In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import copy
import os
import time
from os import path as osp
import numpy as np
import torch
import yaml
from PIL import Image
import cv2
%matplotlib inline
from matplotlib import pyplot as plt

from sacred import Experiment
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from tqdm.notebook import tqdm
from tracktor.config import get_output_dir
from tracktor.datasets.factory import Datasets
from tracktor.datasets.custom_wrapper import CustomSequence
from tracktor.frcnn_fpn import FRCNN_FPN
from tracktor.oracle_tracker import OracleTracker
from tracktor.reid.resnet import ReIDNetwork_resnet50
from tracktor.tracker import Tracker
from tracktor.utils import (evaluate_mot_accums, get_mot_accum,
                            interpolate_tracks, plot_sequence)
from torchvision.ops.boxes import clip_boxes_to_image, nms

In [3]:
def get_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    #img = cv2.resize(img, (512, 512))
    return img

def hist_eq(img):
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max()/ cdf.max()
    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    return cdf[img]

def print_img(img):
    plt.figure(figsize=(15, 15))
    plt.imshow(img, cmap='Greys_r')
    
def draw_bounding(img, x1, x2, y1, y2, label):
    # draw a rectangle on the image
    h, w = img.shape[:2]
    cv2.putText(img, label , (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,0,255), 2)
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), 2)

In [5]:
seq_name = 'bækvej_faxe'
img_dir = "../../data/" + seq_name
file_names = sorted(os.listdir(img_dir), key = lambda x: x[:4])
file_path = [img_dir  + "/" + name for name in file_names]

In [6]:
def IoU(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [7]:
IoU((1,1,4,4),(3,2,5,3))

0.1

In [39]:
obj_detect_model = "../../output/custom/model_epoch_30.model"
obj_detect = FRCNN_FPN(num_classes=31)
obj_detect.load_state_dict(torch.load(obj_detect_model, map_location=lambda storage, loc: storage))
obj_detect.eval()
obj_detect.cuda();
transforms = ToTensor()

track_count = {}
track_score = {}
track_label = {}
available = {}
curr_track_id = 0
all_boxes = [{}]

for i in tqdm(range(len(file_path))):
    img = Image.open(file_path[i]).convert("RGB")
    img_tensor = transforms(img).unsqueeze(0)
    obj_detect.load_image(img_tensor)
    
    prev_frame = all_boxes[-1]
    
    stage = {}
    remove = set()
    
    this_frame = {}
    
    if len(prev_frame) != 0:
        box_list = [prev_frame[track_id].unsqueeze(0) for track_id in prev_frame]
        label_list = [track_label[track_id].unsqueeze(0) for track_id in prev_frame]
        boxes = torch.cat(box_list, 0)
        labels = torch.cat(label_list, 0)
        
        new_boxes, new_scores = obj_detect.predict_boxes(boxes, labels, -1)
        new_boxes = clip_boxes_to_image(new_boxes, img_tensor.shape[-2:])
        
        track_num = 0
        for track_id in prev_frame:
            stage[track_id] = new_boxes[track_num]
            track_score[track_id] = new_scores[track_num]
            if new_scores[track_num] < 0.01:
                remove.add(track_id)
            track_num += 1
                
                
    det_boxes, det_labels, det_scores = obj_detect.detect(img_tensor, -1)
    det_boxes = clip_boxes_to_image(det_boxes, img_tensor.shape[-2:])
    keep = nms(det_boxes, det_scores, 0.1).cuda()
    taken = set()
    
    # back up the tracks
    for track_id in stage:
        track_box = stage[track_id]
        consider = []
        for det_id in keep:
            det_box, det_label, det_score = det_boxes[det_id], det_labels[det_id], det_scores[det_id]
            if (det_score > 0.4 and 
                det_score > track_score[track_id] and
                IoU(det_box, track_box) > 0.2 and
                det_label == track_label[track_id]):
                    
                taken.add(det_id)
                consider.append(det_id)
                    
        if len(consider) != 0:
            high_ind = max(consider,
                           key = lambda x: det_scores[x])#IoU(track_box, det_boxes[x]))
            if track_id in remove:
                remove.remove(track_id)
            stage[track_id] = det_boxes[high_ind]
            track_score[track_id] = det_scores[high_ind]
        if len(taken) == len(keep):
            break
                
    # initialize new tracks
    for det_id in keep:
        if det_id not in taken:
            det_box, det_label, det_score = det_boxes[det_id], det_labels[det_id], det_scores[det_id]
            toAdd = True
            for track_id in stage:
                track_box = stage[track_id]
                if(IoU(track_box, det_box) > 0.1):
                    toAdd = False                   
            if det_score > 0.4 and toAdd:
                new_track = curr_track_id
                curr_track_id += 1

                track_label[new_track] = det_label
                stage[new_track] = det_box
                track_score[new_track] = det_score
                track_count[new_track] = 0
                
    # Transfer from stage to the array
    for track_id in stage:
        if track_id not in remove:
            this_frame[track_id] = stage[track_id]
            track_count[track_id] += 1
            
    all_boxes.append(this_frame)

In [40]:
# This is to delete the tracks with less than 10 frames
re_id = 0
renumeration = {}
for frame in all_boxes:
    for track_id in frame:
        if track_count[track_id] > 10 and track_id not in renumeration:
            renumeration[track_id] = re_id
            re_id += 1
print("done")

done


In [44]:
# Record the video
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter("tracking.mp4", fourcc, 30, (1024, 1024), True)

i = 0
for frame in tqdm(all_boxes[1:]):
    cv_img = get_img(file_path[i])
    i+= 1
    output_img = cv_img.copy()
    for track_id in frame:
        if track_id in renumeration:
            box = frame[track_id]
            label = frame[track_id]
            score = frame[track_id]
            score = score.cpu().numpy()
            label = label.cpu().numpy()

            image_text = str(track_id) + " " + str(score) 
            draw_bounding(output_img, box[0], box[2], box[1], box[3], str(track_id))
    
    writer.write(output_img)     
writer.release()